# Image Classification 1

In [4]:
# imports
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D

In [5]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True

## The data shuffled and Split between trian and test sets

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape: ', x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170498071/170498071 [==============================] - 4s 0us/step
x_train shape:  (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
model = Sequential()

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same',
                input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(48, (3,3), padding='same',
                input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(80, (3,3), padding='same',
                input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3,3), padding='same',
                input_shape=x_train.shape[1:]))
model.add(GlobalMaxPooling2D())
model.add(Dropout(0.25))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.Adam(lr=0.0001)

# let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
             optimizer=opt,
             metrics=['Accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 48)        13872     
                                                                 
 activation_1 (Activation)   (None, 32, 32, 48)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 48)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 48)        0         
                                                      

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
tbCallBack = keras.callbacks.TensorBoard(log_dir='.Graph2', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
             batch_size=batch_size,
             epochs=epochs,
             validation_data=(x_test, y_test),
             shuffle=True, callbacks=[tbCallBack])
else:
    print("Using real-time data augmentation.")
    """
    datagen = ImageDataGenerator(
        featurewise_center=False, # set input to 0 over the dataset
        samplewise_center=False, # set each sample mean to 0
        featurewise_std_normalization=False, # divide inputs by the std of the dataset
        samplewise_std_normalization=False, # divide each inputs by its standard
        zca_whitening=False, # apply ZCA whitening
        rotation_range=0, # randomly rotate images in the range(degrees 0, 180)
        width_shift_range=0.1, # randomly shift images horizantally(fraction of total width)
        height_shift_range=0.1, # randoemly shift images vertically (fraction of total height)
        horizontal_flip=True, # randomly flip images
        vertical_flip=True) # randomly flip images
    """
    datagen = ImageDataGenerator(
        featurewise_center=False, # set input to 0 over the dataset
        samplewise_center=False, # set each sample mean to 0
        featurewise_std_normalization=False, # divide inputs by the std of the dataset
        samplewise_std_normalization=False, # divide each input by its std
        zca_whitening=False, # apply ZCA whitening
        rotation_range=10, # randomly rotate images in the range(degrees, 0 to 180)
        width_shift_range=0.2, # randomly shift images horizontally(fraction of total width)
        height_shift_range=0.2, # randomly shift images vertically(fraction of the total height)
        horizontal_flip=True, # randomly flip images
        vertical_flip=False) # randomly flip images

    # compute quantities required for featurewise normalization
    # (mean, std and principal components if ZCA whitening is applied)
    datagen.fit(x_train)
    
    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                       steps_per_epoch=x_train.shape[0] // batch_size,
                       epochs=epochs,
                       validation_data=(x_test, y_test), callbacks=[tbCallBack])
        
    

Using real-time data augmentation.
Epoch 1/100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


1562/1562 [==============================] - 332s 212ms/step - loss: 2.0438 - Accuracy: 0.2289 - val_loss: 1.7684 - val_Accuracy: 0.3644
Epoch 2/100
1562/1562 [==============================] - 341s 219ms/step - loss: 1.7667 - Accuracy: 0.3417 - val_loss: 1.6518 - val_Accuracy: 0.3864
Epoch 3/100
1562/1562 [==============================] - 339s 217ms/step - loss: 1.6517 - Accuracy: 0.3932 - val_loss: 1.4749 - val_Accuracy: 0.4623
Epoch 4/100
1562/1562 [==============================] - 345s 221ms/step - loss: 1.5712 - Accuracy: 0.4223 - val_loss: 1.4128 - val_Accuracy: 0.4826
Epoch 5/100
1562/1562 [==============================] - 341s 218ms/step - loss: 1.5093 - Accuracy: 0.4470 - val_loss: 1.3548 - val_Accuracy: 0.5012
Epoch 6/100
1562/1562 [==============================] - 341s 218ms/step - loss: 1.4559 - Accuracy: 0.4698 - val_loss: 1.3172 - val_Accuracy: 0.5160
Epoch 7/100
1562/1562 [==============================] - 337s 215ms/step - loss: 1.4096 - Accuracy: 0.4868 - val_loss: